In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests5

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)

# a  = .25

In [3]:
a = .25
num_params = 1

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.99 & 0.96 & 0.95 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.01 & 0.00 & 0.03 & 0.03 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.01, 0.01]), array([0.99, 0.01, 0.  ]), array([0.96, 0.01, 0.03]), array([0.95, 0.02, 0.03]), array([1., 0., 0.]), 0.06040519467859316, 5.119947798742795, 5.5500403700965215)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.96 & 0.95 & 0.97 & 1.00   \\
Model 1 & 0.01 & 0.02 & 0.03 & 0.02 & 0.00   \\
Model 2 & 0.01 & 0.02 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.98, 0.01, 0.01]), array([0.96, 0.02, 0.02]), array([0.95, 0.03, 0.02]), array([0.97, 0.02, 0.01]), array([1., 0., 0.]), 0.05751697496168026, 7.2557211703375115, 7.568342311489618)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.93 & 0.92 & 0.92 & 0.99   \\
Model 1 & 0.06 & 0.06 & 0.07 & 0.06 & 0.01   \\
Model 2 & 0.02 & 0.01 & 0.01 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.92, 0.06, 0.02]), array([0.93, 0.06, 0.01]), array([0.92, 0.07, 0.01]), array([0.92, 0.06, 0.02]), array([0.99, 0.01, 0.  ]), 1.1078023708169678, 10.86249452827712, 10.871927374721613)


# a  = .125

In [7]:
a = .125
num_params = 1

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.98 & 0.97 & 0.99 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.02 & 0.01 & 0.00   \\
Model 2 & 0.00 & 0.01 & 0.01 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([0.98, 0.01, 0.01]), array([0.97, 0.02, 0.01]), array([0.99, 0.01, 0.  ]), array([1., 0., 0.]), 0.13843454365268815, 2.762154455640189, 2.9725707440968825)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.96 & 0.97 & 0.96 & 1.00   \\
Model 1 & 0.01 & 0.02 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.02 & 0.02 & 0.02 & 0.03 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.01, 0.02]), array([0.96, 0.02, 0.02]), array([0.97, 0.01, 0.02]), array([0.96, 0.01, 0.03]), array([1., 0., 0.]), 0.45031492173643095, 4.029978093105361, 3.8389063489122526)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.95 & 0.96 & 1.00   \\
Model 1 & 0.03 & 0.02 & 0.03 & 0.02 & 0.00   \\
Model 2 & 0.02 & 0.03 & 0.02 & 0.02 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.03, 0.02]), array([0.95, 0.02, 0.03]), array([0.95, 0.03, 0.02]), array([0.96, 0.02, 0.02]), array([1., 0., 0.]), 0.045294825141051084, 5.82123097472161, 5.617957497522666)


# a = 0

In [11]:
a = 0
num_params = 1

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.99 & 0.99 & 0.99 & 1.00 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([1., 0., 0.]), array([1., 0., 0.]), 0.011238503961234865, 1.1702973618919048, 1.2998836659680864)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), 0.024684178326519125, 0.9922257974023282, 1.2734190198559197)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 1.00 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -0.015734031759326972, 0.9487957070424576, 1.30799041130392)
